# Geometry Optimizations and UV-Vis Spectroscopy

In this lab, we will go over two topics. First, we will optimizing the geometry of a
molecule, and then compute its absorbance bands found in the ultraviolet and visible portions of
the electromagnetic spectrum.

## Geometry Optimization

A very important step in may electronic structure computations is to find the *optimum geometry*. That is,
we would like to find the position of the coordinates that minimizes the energy (meaning
we are finding the most stable coordinates). This geometry depends on the level
of theory (method and basis set), although similar methods will often result in nearly-identical
geometries.

Optimizations are typically done by computing the *gradient* of the molecule. The *gradient* is
the first-derivative of energy with respect to coordinates. Then, the optimizer will take a
step in the direction that decreases the energy.

When the gradient is equal to zero (within some tolerance), then the minimum of the
energy is found.

## Optimizations with Psi4

To optimize a molecule with psi4, we first read a molecule

In [ ]:
import psi4
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
import numpy as np

In [ ]:
# open one of the molecule files (in the 'molecules' folder, and read its contents)
with open('molecules/benzene.xyz', 'r') as f:
    mol_data = f.read()
mol = psi4.geometry(mol_data)

We can get the coordinates as a numpy array using the `geometry()` method

In [ ]:
mol_geo = mol.geometry().np

In [ ]:
# Exercise: Calculate the distance between the first two atoms of the molecule
# You will do this a few times, so best to write a function

Now let's optimize the molecule using psi4. We set the output file to prevent the output
being printed here, and then call `psi4.optimize` with our level of theory and molecule. `psi4.optimize` function
will return the energy of the optimized geometry.

**NOTE:** Psi4 will modify the input molecule in-place, rather than return a copy

In [ ]:
psi4.set_output_file("optimize_benzene.txt")
energy = psi4.optimize("pbe/6-31g*", molecule=mol)

In [ ]:
# Exercise - recalculate the distance between the first two atoms of the molecule

### Questions

1. How did the distance change?
2. Try doing the optimization with the b3lyp/def2-svp level of theory. How does the distance change there? And how does the energy compare?

In [ ]:
psi4.set_output_file("optimize_benzene.txt")
energy = psi4.optimize("b3lyp/def2-svp", molecule=mol)

## Calculating a UV-Vis spectrum

What happens in a molecule when it absorbs photons of light depend on the wavelength (and therefore energy) of the photon.
For photons in the ultraviolet (around 100-400nm) and visible (400-700nm) spectrum, absorbance of photons
correspond to excitation of electrons to higher energy levels.

These excitations can be computed using electronic structure theory. This will be done in two stages.
First, we will take our optimized molecule and run a singlpoint computation (just like last lab). Then,
we will *time-dependent self-consistent field theory* (TDSCF) or *time-dependent density-functional theory* (TDDFT)
to compute the excitation energies, and *oscillator strength*. The excitation energies correspond to the wavelength
of light that would be absorbed for that transition, and the oscillator strength is related to the strength of the
absorption (and therefore the height of the peak in a UV-Vis spectrum).

First, we need to tell Psi4 to keep some intermediate quantities that will be needed for the TDSCF step.

In [ ]:
options = {"save_jk": True}
psi4.set_options(options)

Then, we can set the output file and run the singlepoint. We need to tell psi4 the level of theory, molecule, and also that we want
to keep the resulting wavefunction information (orbitals). That will be needed for the TDSCF.

For the TDSCF step, we ask for 10 excited states

In [ ]:
psi4.set_output_file("benzene_excitations.txt")
e, wfn = psi4.energy("pbe/6-31g*", return_wfn=True, molecule=mol)
res = tdscf_excitations(wfn, states=10)

Let's print out what was returned

In [ ]:
# Let's print out what was returned
print(res)

We see it is a list of dictionaries of various properties, each describing an electronic transition

### Tasks

The excitation energies represent the wavelength of photons absorbed for that transition. First, create a list of wavelengths from these energies

To convert these energies to wavelengths, we use the usual formula

$$ E = hc/\lambda $$

where $E$ is the energy , $h$ is Planck's constant, $c$ is the speed of light, and $\lambda$ is the wavelength. You might be easiest to write a function that takes in
an energy in hartree and returns a wavelength in nanometers.

* See [https://physics.nist.gov/cuu/Constants/Table/allascii.txt](https://physics.nist.gov/cuu/Constants/Table/allascii.txt) for conversion constants.
* The excitation energies given from the TDDFT module are in Hartrees (atomic units)
* Generally convert everything to SI units (hartrees to Joules) then use Planck's constant and the speed of light in SI units.


Once you have all the energies converted to wavelength, plot these wavelengths on the x-axis, and `OSCILLATOR STRENGTH (LEN)` on the y-axis as a bar plot. In what regime is benzene most absorbant? Does this match your experience (ie, what color is liquid benzene?)

(The oscillator strength is related to the strength of the absorption. However, we typically only care about the shape of the spectrum and therefore the relative peak heights)

## Other molecules

Azulene is an organic molecule with a dark blue color. It is also small enough that we should be able to compute its UV-Vis spectrum as well.
Do the above procedure, but using the `azulene.xyz` file. Do you see any absorbance in the visible spectrum? What color does that correspond to?